In [1]:
import re
import numpy as np
import pandas as pd

import cufflinks as cf

cf.go_offline()

# Load data

In [2]:
data_file = "TS40.txt"

with open(data_file, "r") as f:
    data_str = f.read()

In [3]:
# space between values in this file is not uniform, make it uniform in the following two lines

# remove newline characters
data_str = data_str.replace("\n", "")

# add space before negative values
data_str = re.sub("(\d)(-)", r"\1 \2", data_str)

# split values into a list
data_list = data_str.split()

# load data into numpy data array
data = np.array(data_list, dtype=np.float32)

# reshape data into 2d array, so each row is a channel
data = data.reshape((200, 1000))

# load data into a pandas dataframe
data = pd.DataFrame(data)

In [4]:
dt = 0.004

# pick one channel (7th) and store it a series
s = data.iloc[6]

# set series index in accordance to dt
s.index = s.index * dt
s.index.name = "time [seconds]"
s.name = "Channel 7"

# Plot channel in time domain

In [5]:
s.iplot(title=s.name, xTitle=s.index.name, yTitle="Amplitude")

# Plot in frequency domain

In [6]:
def fft(s, dt):
    
    """
    """
    # perform fft
    s_hat = np.fft.rfft(s)
    freq = np.fft.rfftfreq(n=s.size, d=dt)

    # store results in a series
    s_hat = pd.Series(s_hat, index=pd.Index(freq, name="Frequency [Hz]"), name=(s.name + " fft"))
    
    return s_hat

In [30]:
s_hat = fft(s, dt)
s_hat.abs().iplot(title=s_hat.name, xTitle=s_hat.index.name, yTitle="Amplitude")

# Filter

In [8]:
def build_filter(s_hat, low, high):
    
    """
    Parameters
    ----------
    s_hat: pandas Series
        FFT of a signal
    
    low, high: float
        Filter limits
    """
    
    ## build the filter ##
    # make a,b and c,d octave apart
    a = low / 2
    b = low
    c = high
    d = high * 2
    
    freq = s_hat.index
    mask1 = (~((freq < a) | (freq > d))).astype(np.int)    # make all number outside trapezoid zero
    mask2 = ((freq > b) & (freq < c)).astype(np.int)    # make all numbers between b and c equal one
    
    # add the 2 together, normalize
    mask = (mask1 + mask2) / 2
    
    # replace all values equal 0.5 with linear slope
    # get indecies of first and last occurences of 1
    ind1 = np.where(mask==1)[0]
    first1, last1 = ind1[0], ind1[-1]

    # get indecies of first and last occurences of 0.5
    ind05 = np.where(mask==0.5)[0]
    first05, last05 = ind05[0], ind05[-1]

    # create arrays for liner slope edges of trapezoid
    left_edge = np.linspace(0, 1, (first1 - first05))
    right_edge = np.linspace(1, 0, (last05 - last1))

    # replace 0.5 values in mask with the edges
    mask[first05:first1] = left_edge
    mask[last1+1:last05+1] = right_edge
    
    # store in series
    freq_filter = pd.Series(mask, index=s_hat.index, name="Filter in frequency domain")
    
    return freq_filter

In [9]:
freq_filter = build_filter(s_hat, low=18, high=22)

# apply the filter in freqeuncy domain by multiply the filter element wise with s_hat
s_hat_filtered = (freq_filter * s_hat).rename("Filtered")

# plot in frequency domain the filter,the fft result and fft after filtering
pd.concat([s_hat.abs(), freq_filter, s_hat_filtered.abs()], axis=1).iplot(xTitle=s_hat.index.name, yTitle="Amplitude")

In [10]:
# perform inverse fft
s_filtered = np.fft.irfft(s_hat_filtered)

# store fitered time series in pandas Series object
s_filtered = pd.Series(s_filtered, index = s.index, name=(s.name + " filtered"))

# plot in time domain filtered signal against original
pd.concat([s, s_filtered], axis=1).iplot(xTitle=s.index.name, yTitle="Amplitude")

In [11]:
# perform inverse fft on filter to time domain representation
time_filter = np.fft.irfft(freq_filter)

# shift to middle
time_filter = np.fft.ifftshift(time_filter)

# apply the filter in time domain by performing a circular convolution with original signal
s_filtered2 = np.convolve(s, time_filter, "same")
s_filtered2 = pd.Series(s_filtered2, name=(s.name + " filtered 2"))
s_filtered2.index = s_filtered2.index * dt

In [12]:
pd.concat([s, s_filtered, s_filtered2], axis=1).iplot(xTitle=s.index.name, yTitle="Amplitude")